# Analysis of the historical price of a troy ounce of gold adjusted for inflation

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from statsmodels.tsa.seasonal import STL

In [ ]:
# Get gold price and US CPI dfs
df_au = pd.read_csv("../data/AU.csv")
df_cpi = pd.read_csv("../data/US_CPI_M.csv")

In [ ]:
# Append the CPI estimate for April of 2024 to the CPI df
new_row = pd.DataFrame({"year_month": ["2024-04"], "rate": [3.4]})
df_cpi = pd.concat([df_cpi, new_row], ignore_index=True)

In [ ]:
# Set date as index for both dfs
for df in df_au, df_cpi:
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df.set_index(date_col, inplace=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

In [ ]:
# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

### Adjustment of gold price for inflation (using 2023 prices)

In [ ]:
# Make CPI rates decimal
df_cpi["rate"] = df_cpi["rate"] / 100

In [ ]:
# Get cumulative product of rates until 2023 (which is the base year)
all_months = []
for month in range(1, 12 + 1):
    # Cumulative product of year over year CPI rate for each month number
    cumulative_product = np.cumprod((1 + df_cpi[:f"2023-{month:02}"]["rate"])[::-12])
    # Shift the cumulative product to align with the prior year
    cumulative_product = cumulative_product.shift(1).fillna(1)
    all_months.append(cumulative_product)

In [ ]:
# Add cumulative CPI rates to the df
df_cpi["cumulative_rate"] = pd.concat(all_months)

In [ ]:
# Get 2024 months cumulative rate by just inverting respective rate
df_cpi.loc[df_cpi.index.year == 2024, "cumulative_rate"] = 1 / (1 + df_cpi["rate"])

In [ ]:
# Add the cumulative CPI rates to the gold price df (May values will have the same rate as April)
df_cpi_reindexed = df_cpi.reindex(df_au.index, method="ffill")
df_au["cpi_cumulative_rate"] = df_cpi_reindexed["cumulative_rate"]

In [ ]:
# Adjust all prices to 2023 dollars
df_au["real_price"] = df_au["price"] * df_au["cpi_cumulative_rate"]

## The price of gold adjusted for inflation across time

In [ ]:
# Get 1-year moving average
df_au["real_price_1y_ma"] = df_au["real_price"].rolling(window=252).mean()

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_au, x=df_au.index, y="real_price", label="Price", color="yellow", linewidth=0.25)
sns.lineplot(data=df_au, x=df_au.index, y="real_price_1y_ma", label="Moving average (1-year)", color="aqua", linewidth=0.75)

plt.title("Price of a troy ounce of gold across time in 2023 dollars")
plt.xlabel("Date")
plt.ylabel("Price")

plt.show()

### Comparison of real price with nominal price

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_au, x=df_au.index, y="real_price", label="Price adjusted for inflation", color="yellow", linewidth=0.4)
sns.lineplot(data=df_au, x=df_au.index, y="price", label="Nominal price", color="violet", linewidth=0.4)

plt.title("Price of a troy ounce of gold across time")
plt.xlabel("Date")
plt.ylabel("Price")

plt.show()

In [ ]:
# All time high
ath_date = df_au["real_price"].idxmax()
df_au.loc[[ath_date], ["real_price"]]

In [ ]:
# All time low
atl_date = df_au["real_price"].idxmin()
df_au.loc[[atl_date], ["real_price"]]

In [ ]:
# Peaks
peaks, _ = signal.find_peaks(df_au["real_price"], distance=800)
df_au.iloc[peaks].nlargest(10, "real_price").sort_values("date")[["real_price"]].T

In [ ]:
# Valleys
valleys, _ = signal.find_peaks(-df_au["real_price"], distance=1000)
df_au.iloc[valleys].nsmallest(10, "real_price").sort_values("date")[["real_price"]].T

In [ ]:
# Real price appreciation since first entry
first_entry_price = df_au.iloc[0]["real_price"]
last_entry_price = df_au.iloc[-1]["real_price"]
(last_entry_price - first_entry_price) / first_entry_price

**Key takeaways:**
- the real price does not rise consistently on long term (15 year +) unlike the equities market
- the price still fluctuated a lot on the long term, but ultimately accompanying inflation (making gold a store of value)
- is it a good inflation hedge? well if since the early 80s, the value is practically the same, although it suffered a lot of fluctuations!
- the all time high now is on 1980.
- there is a huge valley between 80s- 2010s and small valley in 2010s, and an even smaller valley after 2020.
- there is some resistance at 2.5k.
- so far the real price appreciated about 646% since first point.

### Real price change year-over-year

In [ ]:
# Get YoY real returns
# Get yearly gold real price df with first and last prices
df_au_yearly = df_au.groupby(df_au.index.year)["real_price"].agg(
    first_price="first",
    last_price="last"
)
# Get YoY return
df_au_yearly["real_price_change"] = (df_au_yearly["last_price"] - df_au_yearly["first_price"]) / df_au_yearly["first_price"]

In [ ]:
plt.figure(figsize=(14, 8))

sns.barplot(data=df_au_yearly, x=df_au_yearly.index, y="real_price_change", color="gold")

plt.title("Year-over-year real return of gold across time")
plt.xlabel("Year")
plt.ylabel("Return")

# Show only beginning of decade
ax = plt.gca()
for index, label in enumerate(ax.get_xticklabels()):
    if index % 10 - 2 != 0:
        label.set_visible(False)

plt.show()

In [ ]:
# Average YoY real return
df_au_yearly["real_price_change"].mean().round(4)

In [ ]:
# Average YoY real return since 1980
df_au_yearly.loc["1980":]["real_price_change"].mean().round(4)

In [ ]:
# Average YoY real return since 2000
df_au_yearly.loc["2000":]["real_price_change"].mean().round(4)

In [ ]:
# Median YoY real return
df_au_yearly["real_price_change"].median().round(4)

In [ ]:
# Standard deviation YoY real return
df_au_yearly["real_price_change"].std().round(4)

In [ ]:
# Get average, median and standard deviation YoY return per decade
df_au_yearly["decade"] = (df_au_yearly.index // 10) * 10
df_au_yearly.groupby("decade").agg(
    avg_yoy_price_change=("real_price_change", lambda x: x.mean().round(4)),
    median_yoy_price_change=("real_price_change", lambda x: x.median().round(4)),
    std_yoy_price_change=("real_price_change", lambda x: x.std().round(4))
).T

**Key takeaways:**
- as expected the bar chart of the real price return is very similar, only the rates really change.
- the average year-over-year real return of gold is 8.5% which seems a lot.
- the median yoy real return is a bit higher than the nominal counterpart.
- if we check it since the 80s it drops to 4%.
- although since the 2000s it becomes 10%!
- just like in the nominal price, the best decades were the 70s 2000s and 2020s so far.

### STL decomposition (trend, seasonality, and residuals)

In [ ]:
stl = STL(df_au["real_price"], period=252).fit()

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df_au.index, df_au["real_price"], label="Original", color="yellow", linewidth=0.5)
axes[0].set_title("Price of a troy ounce of gold across time in 2023 dollars")

axes[1].plot(df_au.index, stl.trend, label="Trend", color="aqua", linewidth=1)
axes[1].set_title("Trend component")

axes[2].plot(df_au.index, stl.seasonal, label="Seasonal", color="fuchsia", linewidth=0.5)
axes[2].set_title("Seasonal component")

axes[3].plot(df_au.index, stl.resid, label="Residual", color="red", linewidth=0.5)
axes[3].set_title("Residual component")

plt.xlabel("Date")

# plt.tight_layout()
plt.show()

#### Trend analysis

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(stl.trend, color="aqua", linewidth=1)

plt.title("Trend component of the real price of gold across time")
plt.xlabel("Date")
plt.ylabel("Trend")

plt.show()

In [ ]:
# Top 3 peaks
peaks, _ = signal.find_peaks(stl.trend, distance=800)
top_peaks = stl.trend.iloc[peaks].nlargest(3)
pd.DataFrame({"value": top_peaks}).T

**valleys are more clear, judging by trend. Also the 2012 peak is actually higher than 1980 peak.**

#### Seasonality analysis

In [ ]:
seasonal = stl.seasonal
monthly_avgs = seasonal.groupby(seasonal.index.month).mean()

plt.figure(figsize=(14, 8))

plt.plot(monthly_avgs.index, monthly_avgs.values, marker='o', color="fuchsia", linewidth=1)

plt.title("Average of the seasonal component of the real price of gold over the year")
plt.xlabel("Month")
plt.ylabel("Seasonality")

plt.show()

**the average seasonality is different when adjusting for inflation, and looks more like the description of some analysts.**

### No need to check anything related to volatility or short-term price analysis.